In [ ]:
import pandas as pd
import glob
import numpy as np

## Combining multiple data files (which all must have the same columns)

We can use `glob` to select all files in the data directory. Here the `*` character stands for 'match anything' (and not multiplication, as it is used elsewhere). This line says to get all the files ending in `.csv` that are in any subdirectory of `data/`. 

In [ ]:
data_files = glob.glob("data/*/*.csv")
data_files

## Combining the files together

In [ ]:
all_files = []
for file in data_files:
    data_temp = pd.read_csv(file)
    all_files.append(data_temp)

In [ ]:
data = pd.concat(all_files)
data

We also want to rename those columns into things that remove the newlines (or `\n`)

In [ ]:
data.columns

In [ ]:
data.columns = ['year', 'location', 'first_name', 'last_name', 'title', 'total_pay', 'regular_pay', 'overtime_pay', 'other_pay']

## Filter to 2021 data

Because 2021 is a number, we don't need to put single quotes around it.

In [ ]:
data_2021 = data.query("year == 2021")

## Counting the number of workers in each location:
We will do a `groupby` with the location column. After the groupby function, you should usually specify which column you want to work with. We want to count the number of workers in each location, so it doesn't matter which column you want to work with here. It will matter when we do mean, median, or total salary later, we have to specify which salary column we want to use.

In [ ]:
data_2021.groupby('location')['year'].count()

We can sort the list smallest to largest:

In [ ]:
data_2021.groupby('location')['year'].count().sort_values()

We can visualize a horizontal bar graph by adding `.plot(kind='barh')` to the sorted table:

In [ ]:
data_2021.groupby('location')['year'].count().sort_values().plot(kind='barh')

# Cleaning data
## Classifying titles
One problem is that there are many different specialized titles, and the payroll system doesn't make it easy to tell them apart. At UCSD we have [this site](http://hr.ucsd.edu/tpp/) which you can use to lookup payroll titles and see a description.

We will create a new function to classify these:

In [ ]:
def classify_title(text):
    text = str(text).upper()
    
    if "POSTDOC" in text:
        return "POSTDOC"
    elif "GSHIP" in text or "GSR" in text:
        return "GRAD"
    elif "LECT" in text or "TEACHER" in text:
        return "LECTURER"
    elif "DEAN" in text and "ASST" not in text and "AST" not in text:
        return "EXEC"
    elif "VC" in text and "ASST" not in text and "AST" not in text and 'SVC' not in text:
        return "EXEC"
    elif "CHAN" in text and "ASST" not in text and "AST" not in text:
        return "EXEC"
    elif "CMO MED CTR" in text:
        return "EXEC"
    elif "PROF" in text:
        return "PROF"
    elif "STDT" in text or "MGR" in text or "AST" in text or "SRA" in text or "ADMIN" in text or "SPEC" in text or "HR" in text or "ANL" in text:
        return "ADMIN"
    elif "NURSE" in text or "PHYS" in text or "PAT" in text or "MED C" in text or "CLIN" in text or "PHARM" in text:
        return "MEDICAL-OTHER"
    else:
        return "UNKNOWN"

In [ ]:
classify_title("HR GENERALIST 2")

In [ ]:
classify_title("TEACHG ASST-GSHIP")

In [ ]:
classify_title("ASSOC DEAN")

In [ ]:
classify_title("AST TO DEAN")

In [ ]:
classify_title("HS ASST CLIN PROF-HCOMP")

### Using `apply` to classify all titles in the dataset

We want to create a new column called `title_class`. This is where we will save the results of applying that function to the `title` column in our original dataset:

In [ ]:
data['title_class'] = data['title'].apply(classify_title)
data

## Removing those whose total pay for a year is less than $15,000

One big problem is that most people do not join or leave the university on Jan 1st. Most people start in the Fall and end in the Spring, meaning that there are lots of rows from first-year or last-year workers. Because we are interested in what the university is paying workers for a full year, a more robust way to do this is to remove all rows for people making under \\$15,000 a year, the minimum salary for someone working 50% time over 3 quarters. Someone working 30 hours a week at \\$15 an hour for a year would be above this and would be included. Someone who recieves $30,000 a year, but is just paid for one or two quarters would not be included.

One problem with this is that it easily filters out the grad students, most of whose salaries for a full year are less than \\$30,000. However, it does not filter out part-year salaries for those who make more than this. The executive who makes \\$600,000 who joins on Dec 1st of a year would have a reported income of \\$50,000 for that year, which would not be filtered out. This likely means that we are slightly underestimating the income of those who make more than \\$30,000 a year. 

Data cleaning is full of political and value choices!


In [ ]:
data.query("title_class == 'GRAD'")['total_pay'].hist(bins=100, figsize=(12,5))

In [ ]:
data_cleaned = data.query("total_pay > 15000")

In [ ]:
data_2021 = data_cleaned.query("year == 2021")

In [ ]:
data_2021['title_class'].value_counts()

We are also going to save a version of the 2021 dataset that is not filtered out, because later we will look at the total money spent on the system:

In [ ]:
data_2021_all = data.query("year == 2021")

## Pivot tables

For a pivot table, you give it a dataset and tell it what a summarized table should look like. You specify the rows (or index), the columns, the values, and how you want to summarize the data. So we want our rows/index to be the title_class, the columns to be the location, and the values to be total_pay. You should also specify the `aggfunc` or aggregation function: should it give you the mean (average), median, sum (total) of all of those values?

In [ ]:
pivot_median_total_by_loc = pd.pivot_table(data_2021, index='title_class', columns='location', values='total_pay', aggfunc='median')
pivot_median_total_by_loc

In [ ]:
pivot_median_total_by_loc.plot(kind='barh', figsize=(5,10), xlabel="Median total pay in 2021")

We can also change the `aggfunc` to `sum` to add up all the entries in each of our pivot table's rows and columns, instead of finding the median. But first, remember how we removed everyone making less than $15k? We want to put them back, which is why we didn't overwrite the `data` variable with our original dataset:

In [ ]:
data_2021_all = data.query("year == 2021")
data_2021_all

In [ ]:
pivot_sum_total_by_loc = pd.pivot_table(data_2021_all, index='title_class', columns='location', values='total_pay', aggfunc='sum')
pivot_sum_total_by_loc

In [ ]:
pivot_sum_total_by_loc.plot(kind='barh', figsize=(9,9))

We can stack the bar graph to make this easier to visualize the total UC spending for each category:

In [ ]:
pivot_sum_total_by_loc.plot(kind='barh', figsize=(9,9), stacked=True, xlabel="Billions of dollars spent in 2021")

Instead of a pivot table, a `groupby` works if you just want to aggregate across one category:

In [ ]:
data_2021_all.groupby('title_class')['total_pay'].sum()

In [ ]:
data_2021_all.groupby('title_class')['total_pay'].sum().plot(kind='barh')

In [ ]:
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_rows', 100)

In [ ]:
data_2021_all.query("title_class == 'EXEC'").sort_values('total_pay', ascending=False)